In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('resources/train.csv')

print(df.describe())

                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  ...  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000  ...   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726  ...   
std       1.112799    30.202904     20.645407   181.066207   456.098091  ..

In [3]:
colonnes = df.columns.tolist()
colonnes.remove('Id')
colonnes.remove('SalePrice')
print(colonnes)

['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fen

In [4]:
df = df.fillna('NO')

In [5]:
min_price = df.SalePrice.min()
print('Prix minimal',min_price)
max_price = df.SalePrice.max()
print('Prix maximal',max_price)

Prix minimal 34900
Prix maximal 755000


In [6]:
fourchette1 = 100000
fourchette2 = 200000
fourchette3 = 300000
fourchette4 = 400000
fourchette5 = 500000
fourchette6 = 600000
fourchettes = [min_price, fourchette1, fourchette2, fourchette3, fourchette4, fourchette5, fourchette6, max_price]

In [7]:
def afficher_valeurs_frequentes(colonne):
    hashmap = {}
    for index,fourchette in enumerate(fourchettes):
        if index < (len(fourchettes)-1):

            prix_min = fourchettes[index]  
            prix_max = fourchettes[index+1] 
            price_filtered_lots = df[(df['SalePrice'] >= prix_min) & (df['SalePrice'] <= prix_max)]

            comptage_valeurs = price_filtered_lots[colonne].value_counts()
            #print(comptage_valeurs)
            
            most_encoutered_value = comptage_valeurs.idxmax()
            if type(most_encoutered_value) in (int, float, complex, np.int64):
                #print('Valeur numérique on va calculer la moyenne')
                most_encoutered_value = comptage_valeurs.mean()
            second_encoutered_value = 'Aucune'
            if len(comptage_valeurs) > 1:
                comptage_valeurs = comptage_valeurs.drop(comptage_valeurs.idxmax())
                second_encoutered_value = comptage_valeurs.idxmax()
            
            hashmap[prix_min,'-',prix_max] = most_encoutered_value
            #print("La valeur la plus fréquente pour la colonne", colonne, "lorsque le prix est entre", prix_min, "et", prix_max, "est :", most_encoutered_value, 'suivie de : ', second_encoutered_value)
    return hashmap

In [8]:
def score_hashmap(hashmap):
    score = 0
    hashmap_keys_values = list(hashmap.values())
    for index in range(len(hashmap_keys_values)):
        percent = 0.80
        if index == 0 :
            #Faire pour les valeurs numérique un score selon valeur proche ou pas.
            if type(hashmap_keys_values[0]) in (int, float, complex, np.int64):
                print(hashmap_keys_values[0], '-', hashmap_keys_values[3])
                if abs(hashmap_keys_values[0] - hashmap_keys_values[3]) < percent  * hashmap_keys_values[0] :
                    score += 1
                    if abs(hashmap_keys_values[3] - hashmap_keys_values[5]) < percent  * hashmap_keys_values[0] : score +=2
                    if abs(hashmap_keys_values[3] - hashmap_keys_values[6]) < percent  * hashmap_keys_values[0] : score +=2
                if abs(hashmap_keys_values[0] - hashmap_keys_values[5]) < percent  * hashmap_keys_values[0] : score += 2
                if abs(hashmap_keys_values[0] - hashmap_keys_values[6]) < percent  * hashmap_keys_values[0] : score += 5
                    
            if type(hashmap_keys_values[0]) not in (int, float, complex, np.int64):
                if hashmap_keys_values[0] != hashmap_keys_values[3]:
                    score += 1
                    if hashmap_keys_values[3] != hashmap_keys_values[5]:score += 2
                    if hashmap_keys_values[3] != hashmap_keys_values[6]:score += 2
                if hashmap_keys_values[0] != hashmap_keys_values[5]:score += 2
                if hashmap_keys_values[0] != hashmap_keys_values[6]:score += 5
            
            
    return score

In [9]:
colonnes_pertinentes = {}
for colonne in colonnes:
    if score_hashmap(afficher_valeurs_frequentes(colonne)) >= 12 : colonnes_pertinentes[colonne] = score_hashmap(afficher_valeurs_frequentes(colonne))
#print(colonnes_pertinentes)

sorted_colonnes_pertinentes = sorted(colonnes_pertinentes.items())
print(sorted_colonnes_pertinentes)

[('1stFlrSF', 12), ('2ndFlrSF', 12), ('3SsnPorch', 12), ('BedroomAbvGr', 12), ('BsmtFinSF1', 12), ('BsmtFinSF2', 12), ('BsmtFullBath', 12), ('BsmtHalfBath', 12), ('BsmtUnfSF', 12), ('EnclosedPorch', 12), ('ExterQual', 12), ('Fireplaces', 12), ('FullBath', 12), ('GarageArea', 12), ('GarageCars', 12), ('GarageYrBlt', 12), ('GrLivArea', 12), ('HalfBath', 12), ('KitchenAbvGr', 12), ('KitchenQual', 12), ('LotFrontage', 12), ('LowQualFinSF', 12), ('MSSubClass', 12), ('MasVnrArea', 12), ('MiscVal', 12), ('MoSold', 12), ('Neighborhood', 12), ('OpenPorchSF', 12), ('OverallCond', 12), ('OverallQual', 12), ('PoolArea', 12), ('ScreenPorch', 12), ('TotRmsAbvGrd', 12), ('TotalBsmtSF', 12), ('WoodDeckSF', 12), ('YearBuilt', 12), ('YearRemodAdd', 12), ('YrSold', 12)]


In [10]:
afficher_valeurs_frequentes('WoodDeckSF')

{(34900, '-', 100000): 4.92,
 (100000, '-', 200000): 4.688775510204081,
 (200000, '-', 300000): 3.076923076923077,
 (300000, '-', 400000): 1.5535714285714286,
 (400000, '-', 500000): 1.1875,
 (500000, '-', 600000): 1.25,
 (600000, '-', 755000): 1.0}